In [8]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

scala_version = '2.12'
spark_version = '3.5.0'
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.6.0'
]
spark = SparkSession.builder.master('local').appName('kafka-example').config('spark.jars.packages', ','.join(packages)).getOrCreate()
spark

In [36]:
from kafka import KafkaConsumer
import time
from pyspark.ml.regression import LinearRegressionModel
from pyspark.sql.types import * 
from pyspark.ml.feature import VectorAssembler, StandardScalerModel

kafka_topic_name = 'turtorial15'
kafka_bootstrap_servers = 'localhost:9092'

consumer = KafkaConsumer(kafka_topic_name, bootstrap_servers=kafka_bootstrap_servers)

model_path = "./linear_regression_model"
linearModel = LinearRegressionModel.load(model_path)

# Define the schema
schema = StructType([
               StructField('Item_Identifier', StringType(), True),
               StructField('Item_Weight', FloatType(), True),
               StructField('Item_Fat_Content', StringType(), True),
               StructField('Item_Visibility', FloatType(), True),
               StructField('Item_Type', StringType(), True),
               StructField('Item_MRP', FloatType(), True),
               StructField('Outlet_Identifier', StringType(), True),
               StructField('Outlet_Establishment_Year', IntegerType(), True),
               StructField('Outlet_Size', StringType(), True),
               StructField('Outlet_Location_Type', StringType(), True),
               StructField('Outlet_Type', StringType(), True),
               StructField('Time_Interval', TimestampType(), True),
              ])

for message in consumer:
    # Extract the message from the ConsumerRecord object
    message_values = message.value
    print(message_values)

    # Replace null values with a default value
    message_values = ['0.0' if x.isNull() else x for x in message_values]
    
    # Create a Row based on the schema
    row = [message_values[i] for i in range(len(schema))]
    
    # Create a DataFrame from the Row
    row = Row(*message_values)
    df = spark.createDataFrame([row], schema=schema)

    df.show(truncate=False)

b'{"Item_Identifier":"NCW18","Item_Weight":15.1,"Item_Fat_Content":"Low Fat","Item_Visibility":0.0,"Item_Type":"Household","Item_MRP":235.5248,"Outlet_Identifier":"OUT045","Outlet_Establishment_Year":2002,"Outlet_Size":null,"Outlet_Location_Type":"Tier 2","Outlet_Type":"Supermarket Type1","time_interval":"2023-12-29 15:35:43"}'


AttributeError: 'int' object has no attribute 'isNull'